# WiNDC National Model

You will need to point to the correct locations for a few objects, listed below

In [12]:
parameter_path = raw"C:\Users\mphillipson\Documents\Modeling\MPSGE_dev_testing\nationaldata_julia\DAAData.jld2"
set_path = raw"C:\Users\mphillipson\Documents\Modeling\MPSGE_dev_testing\nationaldata_julia\Indices.jld2";

In [13]:
using MPSGE_MP

using CSV, DataFrames, JLD2

using JuMP

In [14]:
P= load(parameter_path)["data"] # load in date from saved Notebook output Dict, named P
S= load(set_path)["data"] # load in date from saved Notebook output Dict, named P


I = [i for i∈S[:i] if i∉[:use,:oth]]
J = [i for i∈S[:j] if i∉[:use,:oth]]
VA = [va for va∈S[:va] if va!=:othtax]
FD = S[:fd]
TS = S[:ts]
YR = S[:yr]
M = S[:m]

a_0 = P[:a_0]
id_0 = P[:id_0]
ys_0 = P[:ys_0]
tm_0 = P[:tm_0]
va_0 = P[:va_0]
md_0 = P[:md_0]
fd_0 = P[:fd_0]
m_0 = P[:m_0]
ty_0 = P[:ty_0]
ms_0 = P[:ms_0]
bopdef_0 = P[:bopdef_0]
x_0 = P[:x_0]
ta_0 = P[:ta_0]
#s_0 = P[:s_0]
fs_0 = P[:fs_0]
y_0 = P[:y_0];



1;

In [15]:
yr = Symbol(2017)

WiNnat = MPSGEModel()

#y_ = [j for j∈J if sum(ys_0[yr,j,i] for i∈I) !=0]
#a_ = [i_ for i_∈I if a_0[yr,i_]!=0]


@parameters(WiNnat, begin
    ta, ta_0[yr,J], (index = [J],)
    ty, ty_0[yr,J], (index = [J],)
    tm, tm_0[yr,J], (index = [J],)
end)

@sectors(WiNnat,begin
    Y, (index = [J], description = "Sectoral Production")
    A, (index = [I], description = "Armington Supply")
    MS, (index = [M],description = "Margin Supply")
end)

@commodities(WiNnat,begin
    PA, (index = [I],  description = "Armington Price")
    PY, (index = [J],  description = "Supply")
    PVA, (index = [VA],description = "Value-added")
    PM, (index = [M],  description = "Foreign Exachange")
    PFX
end)

@consumer(WiNnat, RA, description = "Representative Agent")


for j∈J
    add_production!(WiNnat, Y[j],
        ScalarNest(:t; elasticity = 0, children = [
            ScalarOutput(PY[i],ys_0[yr,j,i]; taxes = [Tax(RA,ty[j])]) for i∈I
        ]),
        ScalarNest(:s; elasticity = 0, children = [
            [ScalarInput(PA[i], id_0[yr,i,j]) for i∈I]; 
            [ScalarNest(:va; elasticity =  1, children = 
                [ScalarInput(PVA[va], va_0[yr,va,j]) for va∈VA])
            ];
        ])
    )
end


for m∈M
    add_production!(WiNnat, MS[m],
        ScalarNest(:t; elasticity = 0, children = [
            ScalarOutput(PM[m], sum(ms_0[yr,i,m] for i∈I))
        ]),
        ScalarNest(:s; elasticity = 0, children = [
            ScalarInput(PY[i], ms_0[yr,i,m]) for i∈I
        ])
    )
end


for i∈I
    add_production!(WiNnat, A[i],
        ScalarNest(:t; elasticity=2, children = [
            [ScalarOutput(PA[i], a_0[yr,i], taxes=[Tax(RA,ta[i])],reference_price=1-ta_0[yr,i])];
            [ScalarOutput(PFX, x_0[yr,i])]
        ]),
        ScalarNest(:s; elasticity=0, children = [
            [ScalarInput(PM[m], md_0[yr,m,i]) for m∈M];
            ScalarNest(:dm; elasticity = 2, children = [
                ScalarInput(PY[i], y_0[yr,i]),
                ScalarInput(PFX, m_0[yr,i], taxes = [Tax(RA,tm[i])],reference_price=1+tm_0[yr,i])
            ])
        ])
    )
end


add_demand!(WiNnat, RA,
    [ScalarDem(PA[i], fd_0[yr,i,:pce]) for i∈I],
    [
        [ScalarEndowment(PY[i], fs_0[yr,i]) for i∈I];
        [ScalarEndowment(PFX, bopdef_0[yr])];
        [ScalarEndowment(PA[i], -sum(fd_0[yr,i,xfd] for xfd∈FD if xfd!=:pce)) for i∈I];
        [ScalarEndowment(PVA[va], sum(va_0[yr,va,j] for j∈J)) for va∈VA]
    ]
);

build!(WiNnat);

# Benchmark 

In [16]:
solve!(WiNnat; :cumulative_iteration_limit => 0)

df_benchmark = generate_report(WiNnat);

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_D89.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 292

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 4.1722e-12           I 0.0e+00 1.8e-12 (f[  503])

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 4.172191e-12
Postsolved residual: 4.1722e-12


# Counterfactual

In [17]:
fix(RA,12453.896315446877)

set_value!(ta,0)
set_value!(tm,0)

solve!(WiNnat; :cumulative_iteration_limit => 10_000)

df = generate_report(WiNnat);

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_E16.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Zero:     3 Single:     0 Double:     0
Preprocessed size   : 288

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             3.2840e+02             0.0e+00 (f[  299])
    1     1     0   288 3.9192e+00  1.0e+00    3.3e+00 (f[  299])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 3.9192e+00           I 3.9e-02 8.8e-01 (f[  299])
    1     1     3     3 9.3103e-02  1.0e+00 SO 1.6e-02 5.6e-02 (f[  505])
    2     1     4     4 5.5702e-05  1.0e+00 SO 6.3e-03 4.3e-05 (f[  260])
    3     1     5     5 1.3248e-08  1.0e+00 SO 5.6e-06 1.1e-08 (f[  331])

Major Iterations. . . . 3
Minor Iterations. . . . 3

Postsolved residual: 1.3248e-08


In [18]:
df |>
    x -> sort(x, :margin, rev=true)

Row,var,value,margin
,GenericV…,Float64,Float64
1,PA[pip],0.94824,1.11201e-8
2,RA,12453.9,8.3528e-9
3,A[pip],0.999342,3.60385e-9
4,PVA[surplus],0.948073,3.72893e-11
5,PA[hou],0.948065,1.86446e-11
6,PA[alt],0.947923,1.79057e-11
7,PA[hos],0.947955,1.22782e-11
8,PA[res],0.947895,1.03455e-11
9,PA[pub],0.947905,6.99174e-12
